In [ ]:
from google.colab.patches import cv2_imshow
import cv2

def canny_edge_detection(image, low_threshold, high_threshold):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    cv2.imwrite('grayimg.png', gray)
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (11,11), 0,0)

    cv2.imwrite('blurimg.png', blurred)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, low_threshold, high_threshold)



    return edges

# Read image
image = cv2.imread('/content/2012-10-111.jpg')

# Define thresholds
low_threshold = 50
high_threshold = 180

# Apply Canny edge detection
edges = canny_edge_detection(image, low_threshold, high_threshold)

# Display the result
cv2.imwrite('detectedEdges.png', edges)


True

In [ ]:
import numpy as np
# Apply Hough Line Transform
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=60, minLineLength=10, maxLineGap=25)

# Draw detected lines on a blank image
height, width = edges.shape[:2]
blank_image = np.zeros((height, width, 3), dtype=np.uint8)

if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]

        cv2.line(blank_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Display the result
cv2.imwrite('detectedLines.png', blank_image)


True

In [ ]:
zero_slope_lines=[]
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
                # Calculate slop
        slope = (y2 - y1) / (x2 - x1) if (x2 - x1) != 0 else np.inf
        if slope >=0 and slope<1:
          zero_slope_lines.append(line)



for line in zero_slope_lines:
  x1, y1, x2, y2 = line[0]
  cv2.line(blank_image, (x1, y1), (x2, y2), (0, 0, 255), 2)



# Display the result
cv2.imwrite('detectedLineswithzeroslope.png', blank_image)



True

In [ ]:
# Extract x-coordinates of endpoints of lines
x_coords = []
for line in zero_slope_lines:
  x1, _, x2, _ = line[0]
  x_coords.append(x1)
  x_coords.append(x2)

# Sort x-coordinates
x_coords.sort()
print(x_coords)
# Identify main parking lanes
main_parking_lanes = []
if len(x_coords) > 0:
    main_parking_lanes.append(x_coords[0])
    for i in range(1, len(x_coords)):
        if x_coords[i] - x_coords[i-1] > 20:  # Assume a gap of at least 10 pixels indicates a new lane
            main_parking_lanes.append(x_coords[i])

# Divide the space between each pair of detected main parking lanes into equal segments
parking_spots = []
for i in range(len(main_parking_lanes) - 1):
    start_x = main_parking_lanes[i]
    end_x = main_parking_lanes[i + 1]
    num_segments = 3  # Adjust the number of segments as needed
    segment_width = (end_x - start_x) // num_segments
    for j in range(num_segments):
        parking_spots.append((start_x + j * segment_width, start_x + (j + 1) * segment_width))

# Save the coordinates of each parking spot
with open('parking_spot_coordinates.txt', 'w') as file:
    for idx, spot in enumerate(parking_spots):
        file.write(f'Parking spot {idx + 1}: {spot}\n')
        # Draw rectangles around parking spots on the image
        cv2.rectangle(image, (spot[0], 170), (spot[1], 210), (0, 255, 0), 2)
        cv2.rectangle(image, (spot[0], 135), (spot[1], 165), (0, 255, 0), 2)
        cv2.rectangle(image, (spot[0], 245), (spot[1], 275), (0, 255, 0), 2)
        cv2.rectangle(image, (spot[0], 290), (spot[1], 320), (0, 255, 0), 2)
        cv2.rectangle(image, (spot[0], 370), (spot[1], 420), (0, 255, 0), 2)

# Save the result image
cv2.imwrite('parking_spots_marked.png', image)

# Read image
img = cv2.imread('/content/2012-10-111.jpg')
i = 1
# Open the text file containing pairs of x coordinates
with open('/content/parking_spot_coordinates.txt', 'r') as file:
    for line in file:
        values_str = line.split("(")[1].split(")")[0]
        x1, x2 = map(int, values_str.split(","))

        # Set default values for y-coordinate and dimensions
        y = 170  # Default y-coordinate
        width = abs(x2 - x1)  # Width is the difference between x2 and x1
        height = 40  # Default height

        # Extract the region of interest (ROI)
        roi = img[y:y+height, x1:x2]

        # Save the extracted ROI as an image
        if width>13:
          cv2.imwrite('empty_' + str(i) + '.png', roi)  # Save as 'empty_1.png', 'empty_2.png', etc.
        i += 1

# Destroy all OpenCV windows
cv2.destroyAllWindows()




[41, 89, 90, 109, 130, 180, 180, 221, 259, 286, 314, 322, 383, 429, 433, 474, 530, 541]


In [ ]:
# Read image
img = cv2.imread('/content/2012-09-14_17_36_31.jpg')
i = 1
# Open the text file containing pairs of x coordinates
with open('/content/parking_spot_coordinates.txt', 'r') as file:
    for line in file:
        values_str = line.split("(")[1].split(")")[0]
        x1, x2 = map(int, values_str.split(","))

        # Set default values for y-coordinate and dimensions
        y = 170  # Default y-coordinate
        width = abs(x2 - x1)  # Width is the difference between x2 and x1
        height = 40  # Default height

        # Extract the region of interest (ROI)
        roi = img[y:y+height, x1:x2]

        # Save the extracted ROI as an image
        if width>13:
          cv2.imwrite('empty_' + str(i) + '.png', roi)  # Save as 'empty_1.png', 'empty_2.png', etc.
        i += 1

# Destroy all OpenCV windows
cv2.destroyAllWindows()

TypeError: 'NoneType' object is not subscriptable